## Script info

In [ ]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""   Analyse simulations   -- Version 1.0
Last edit:  2023/08/29
Authors:    Leone, Riccardo (RL)
Notes:      - Analyse all the models
            - Release notes:
                * Initial commit
To do:      - 
Comments:   

Sources: 
"""

## Imports

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import string
from petTOAD_setup import *
from statannotations.Annotator import Annotator

## Directories

In [ ]:
SIM_DIR = RES_DIR / "final_simulations"
A_DIR = SIM_DIR / "a-weight_ws_-0.1-0.1_bs_-0.05-0.05"
A_RAND_DIR = SIM_DIR / "a-weight_ws_-0.1-0.1_bs_-0.05-0.05_random"
G_DIR = SIM_DIR / "G-weight_ws_-1.0-1.0_bs_-0.5-0.5"
G_RAND_DIR = SIM_DIR / "G-weight_ws_-1.0-1.0_bs_-0.5-0.5_random"
SC_DIR = SIM_DIR / "sc_disconn"
HET_DIR = SIM_DIR / "heterogeneous_ws_-0.1-0.1_bs_-0.05-0.05"
HET_RAND_DIR = SIM_DIR / "heterogeneous_ws_-0.1-0.1_bs_-0.05-0.05_random"
FIG_DIR = SIM_DIR / "Figures"
if not Path.exists(FIG_DIR):
    Path.mkdir(FIG_DIR)
MCI_DIR = RES_DIR / "model_simulations" / "MCI_noWMH"

## Functions

In [ ]:
def annotate_star(tbl):
    """
    This function is just to plot an asterisk at the best heatmap value for better recognition.
    """
    star = tbl.where(tbl == tbl.values.max())
    star = star.replace({np.nan: ""})
    star = star.replace({tbl.values.max(): "*"})
    return star


def find_max_b_w_tbl(tbl):
    """
    This function finds the w and b that are associated with the star (best value) in the heatmap of group comparisons
    """
    max_value = tbl.values.max()
    mask = tbl == max_value
    column_names = tbl.columns[mask.any(axis=0)]
    column_floats = [float(col) for col in column_names]
    row_names = tbl.index[mask.any(axis=1)]
    row_floats = [float(row) for row in row_names]
    return row_floats[0], column_floats[0]


def get_best_dfs(big_df, fc_name, phfcd_name, comp_score_name, wmh_dict):
    big_df["comp_score"] = big_df["fc_pearson"] + 1 * big_df["phfcd_ks"]
    # Get the best model fits for fc, phfcd for each subject and create one single df (skip fcd for now)
    res_df_best = pd.DataFrame(
        {
            fc_name: big_df.groupby(["PTID"])[fc_name].max(),
            phfcd_name: big_df.groupby(["PTID"])[phfcd_name].max(),
            comp_score_name: big_df.groupby(["PTID"])[comp_score_name].max(),
        }
    ).reset_index()

    res_df_best["wmh_load"] = [wmh_dict[subj] for subj in res_df_best["PTID"]]
    # This df is to check what are the best w, b in case of the best fc, best phfcd and best glob score
    # First we group by each patient and get the max fc pearson for each
    # Then we merge this df (with the name of pt and max fc) with the df containing also b and w
    # so we have: max(FC), b, w for each patient etc.
    max_fc_vals = big_df.groupby(["PTID"])[fc_name].max().reset_index()
    merged_df_max_fc = pd.merge(big_df, max_fc_vals, on=["PTID", fc_name])
    # We do the same for 1-phFCD
    max_phfcd_vals = big_df.groupby(["PTID"])[phfcd_name].max().reset_index()
    merged_df_max_phfcd = pd.merge(big_df, max_phfcd_vals, on=["PTID", phfcd_name])
    # We do the same for GFS
    max_comp_score_vals = big_df.groupby(["PTID"])[comp_score_name].max().reset_index()
    merged_df_max_comp_score = pd.merge(big_df, max_comp_score_vals, on=["PTID", comp_score_name])


    return merged_df_max_fc, merged_df_max_phfcd, merged_df_max_comp_score


def plot_parm_best_wmh_relationship(parm, df_max_comp_score):
    """
    This function plots the relationship between the best achieved FC and (1-phFCD) and WMH (normalized) in the parameter space that was explored
    """
    fig, axs = plt.subplots(2, 2, figsize=(7, 7))
    # Values that give the best FC
    sns.regplot(
        ax=axs[0, 1],
        data=df_max_comp_score[df_max_comp_score["PTID"].isin(CN_WMH)],
        x="wmh_load",
        y=parm,
        label="CU WMH",
    )
    sns.regplot(
        ax=axs[0, 1],
        data=df_max_comp_score[df_max_comp_score["PTID"].isin(MCI_WMH)],
        x="wmh_load",
        y=parm,
        label="MCI WMH",
    )
    fig.legend()
    fig.tight_layout()


def create_pivot_tables(df_res):
    # Convert the result df into a pivot table so to plot heatmap
    table_fc = pd.pivot_table(
        df_res, values="fc_pearson", index="b", columns="w"
    ).astype(float)
    # Skip fcd for now
    # table_fcd = pd.pivot_table(res_df, values='fcd_ks', index='b', columns='w').astype(float)
    table_phfcd = pd.pivot_table(
        df_res, values="phfcd_ks", index="b", columns="w"
    ).astype(float)
    # Create a composite score by summing up the single model fits so to choose when FC and phFCD have different "best" b-w combinations
    table_comp_score = table_fc + 1 * table_phfcd
    return table_fc, table_phfcd, table_comp_score


def plot_save_heatmaps(table_fc, table_phfcd, table_comp_score, random):
    """
    This function starts from the results dataframe, converts it into a pivot-table, which is what seaborn need to plot the heatmap.
    It does so both for fc and for phfcd. Also plots and saves the results.
    """
    # Create Heatmaps
    fig, axs = plt.subplots(1, 3, figsize=(10, 4))
    sns.heatmap(
        ax=axs[0],
        data=table_fc,
        annot=annotate_star(table_fc),
        fmt="",
        annot_kws={"size": 10},
    )
    axs[0].set_title(f"FC", fontsize=9)

    sns.heatmap(
        ax=axs[1],
        data=table_phfcd,
        annot=annotate_star(table_phfcd),
        fmt="",
        annot_kws={"size": 10},
    )
    axs[1].set_title(f"1 - phFCD", fontsize=9)
    sns.heatmap(
        ax=axs[2],
        data=table_comp_score,
        annot=annotate_star(table_comp_score),
        fmt="",
        annot_kws={"size": 10},
    )
    axs[2].set_title(f"GFS", fontsize=9)
    if not random:
        savename = f"weights_and_bias_heatmap.png"
        fig.suptitle("Random = False")
    else:
        savename = f"weights_and_bias_heatmap_random.png"
        fig.suptitle("Random = True")
    fig.tight_layout()
    plt.savefig(FIG_DIR / savename)


def perform_group_analyses(df_petTOAD, big_df, random):
    """
    This function allow us to select the grouping classification (Fazekas 1, Fazekas 2 or all) as the 'group'variable and then it selects
    only CN and MCI with WMH based on the chosen classification (since we are not interested for this type of model in patients without WMH).
    Then, calls the save_plot_results function, which plots and saves a summary heatmap of the mean FC and 1-phFCD across all patients that fall
    into a category (CN or MCI) according to the chosen grouping.

    Args:
        df_petTOAD:
        big_df:
        group:
        random:

    Returns:
        The b and ws associated with
    """
    CN_WMH = df_petTOAD[df_petTOAD["Group_Fazekas_2.0"] == "CN_WMH"]["PTID"]
    MCI_WMH = df_petTOAD[df_petTOAD["Group_Fazekas_2.0"] == "MCI_WMH"]["PTID"]

    CN_wmh_df = big_df[big_df["PTID"].isin(CN_WMH)]
    CN_wmh_grouped = CN_wmh_df.drop(columns=["PTID"]).groupby(["b", "w"]).mean()
    tbl_fc_CN_wmh, tbl_phfcd_CN_wmh, tbl_comp_score_CN_wmh = create_pivot_tables(
        CN_wmh_grouped
    )
    plot_save_heatmaps(
        tbl_fc_CN_wmh,
        tbl_phfcd_CN_wmh,
        tbl_comp_score_CN_wmh,
        random,
    )

    mci_wmh_df = big_df[big_df["PTID"].isin(MCI_WMH)]
    mci_wmh_grouped = mci_wmh_df.drop(columns=["PTID"]).groupby(["b", "w"]).mean()
    tbl_fc_mci_wmh, tbl_phfcd_mci_wmh, tbl_comp_score_mci_wmh = create_pivot_tables(
        mci_wmh_grouped
    )
    plot_save_heatmaps(
        tbl_fc_mci_wmh,
        tbl_phfcd_mci_wmh,
        tbl_comp_score_mci_wmh,
        random,
    )

    b_CN_wmh, w_CN_wmh = find_max_b_w_tbl(tbl_comp_score_CN_wmh)
    b_mci_wmh, w_mci_wmh = find_max_b_w_tbl(tbl_comp_score_mci_wmh)

    print("The best parameters found for these heatmaps are:")
    print(f"b = {b_CN_wmh}, w = {w_CN_wmh}, for CN with WMH")
    print(f"b = {b_mci_wmh}, w = {w_mci_wmh}, for MCI with WMH")

    return b_CN_wmh, w_CN_wmh, b_mci_wmh, w_mci_wmh


def select_b_w_combo(df_boxplot_long, w1, b1, w2, b2, groupname1, groupname2):
    compare_df1 = df_boxplot_long[
        (df_boxplot_long["w"] == w1) & (df_boxplot_long["b"] == b1)
    ]
    compare_df1 = compare_df1[compare_df1["group"] == groupname1]
    compare_df2 = df_boxplot_long[
        (df_boxplot_long["w"] == w2) & (df_boxplot_long["b"] == b2)
    ]
    compare_df2 = compare_df2[compare_df2["group"] == groupname2]
    joined_compare_df = pd.concat([compare_df1, compare_df2])
    return compare_df1, compare_df2, joined_compare_df


def create_df_boxplot(df_big, best_fit_b, best_fit_w, diagnosis_group_wmh, model_type):
    """
    This function starts from the big database obtained from the model simulation (e.g., a-weighted homogenous, G-weighted homgoeneous, heterogeneous, etc.) and creates
    a smaller dataset with only the observable variables obtained with the best b and w combination as before. It is useful to then create a long df for plotting.
    """
    df = pd.DataFrame()
    df["PTID"] = df_big[
        (df_big["b"] == best_fit_b)
        & (df_big["w"] == best_fit_w)
        & (df_big["Group_Fazekas_2.0"] == diagnosis_group_wmh)
    ]["PTID"]
    df["fc_pearson"] = df_big[
        (df_big["b"] == best_fit_b)
        & (df_big["w"] == best_fit_w)
        & (df_big["Group_Fazekas_2.0"] == diagnosis_group_wmh)
    ]["fc_pearson"]
    df["phfcd_ks"] = df_big[
        (df_big["b"] == best_fit_b)
        & (df_big["w"] == best_fit_w)
        & (df_big["Group_Fazekas_2.0"] == diagnosis_group_wmh)
    ]["phfcd_ks"]
    df["comp_score"] = df["fc_pearson"] + 1 * df["phfcd_ks"]
    df["Model_type"] = [f"{model_type}" for _ in range(len(df))]
    return df


def plot_obs(df, obs, ax):
    """
    This function plots the annotations for statistical comparisons between groups
    """
    order = ["Base", "Homo.", "Hetero.", "G-weight", "Disconn."]
    # Choose the combinations that you want to test..
    pairs = [
        ("Base", "Homo."),
        ("Base", "Hetero."),
        ("Base", "G-weight"),
        ("Base", "Disconn."),
        ("Homo.", "Hetero."),
        ("Homo.", "G-weight"),
        ("Hetero.", "G-weight"),
    ]
    f = sns.boxplot(data=df, x="Model_type", y=obs, ax=ax)
    annotator = Annotator(f, pairs, data=df, x="Model_type", y=obs, order=order)
    # Choose the type of statistical test to perform
    annotator.configure(test="Wilcoxon", text_format="star", loc="inside", verbose=1)
    annotator.apply_and_annotate()
    plt.xticks([0, 1, 2, 3, 4], order)
    plt.xlabel("Model type")


def plot_random_comparison(df, obs, axs):
    """
    This function is for plotting the comparison between the model results achieved in CN and MCI with the same model
    """
    pairs = [
        (("Homo.", "Random"), ("Homo.", "Not Random")),
        (("Hetero.", "Random"), ("Hetero.", "Not Random")),
        (("G-weight", "Random"), ("G-weight", "Not Random")),
    ]

    # This is a dictionary of arguments that are passed into the function inside map_dataframe
    kwargs = {
        "plot_params": {  # this takes what normally goes into sns.barplot etc.
            "x": "Model_type",
            "y": f"{obs}",
            "hue": "Random",
            "hue_order": ["Random", "Not Random"],
        },
        "annotation_func": "apply_test",
        "configuration": {
            "test": "Wilcoxon",
        },  # this takes what normally goes into ant.configure
        "plot": "boxplot",
        # If you want to add the figure into a subplot...
        "ax": axs,
    }

    df["Model_type"] = df["Model_type"].str.replace("Homo. rand.", "Homo.")
    df["Model_type"] = df["Model_type"].str.replace("Hetero. rand.", "Hetero.")
    df["Model_type"] = df["Model_type"].str.replace("G rand.", "G-weight")

    ant = Annotator(None, pairs)
    # We create a FacetGrid and pass the dataframe that we want to use to later apply our functions (plotting the comparisons between groups)
    g1 = sns.FacetGrid(df, aspect=1.5, height=4)
    # map_dataframe accepts a function, which it then applies to the dataframe that is previously passed in the FacetGrid. It also accepts kwargs which
    # are passed inside the function
    g1.map_dataframe(ant.plot_and_annotate_facets, **kwargs)
    plt.close()


def plot_CN_mci_comparison(obs, df_compare_groups): # (obs, axs) if you want subplots
    """
    This function is for plotting the comparison between the model results achieved in CN and MCI with the same model
    """
    df_compare_groups["group"] = df_compare_groups["group"].str.replace("CN", "CU") #Change to CU so that the label is already ok
    pairs = [
        (("Base", "CU WMH"), ("Base", "MCI WMH")),
        (("Homo.", "CU WMH"), ("Homo.", "MCI WMH")),
        (("Hetero.", "CU WMH"), ("Hetero.", "MCI WMH")),
        (("G-weight", "CU WMH"), ("G-weight", "MCI WMH")),
        (("Disconn.", "CU WMH"), ("Disconn.", "MCI WMH")),
    ]

    # This is a dictionary of arguments that are passed into the function inside map_dataframe
    kwargs = {
        "plot_params": {  # this takes what normally goes into sns.barplot etc.
            "x": "Model_type",
            "y": f"{obs}",
            "hue": "group",
            # "hue_order": ["CN WMH", "MCI WMH"],
        },
        "annotation_func": "apply_test",
        "configuration": {
            "test": "Mann-Whitney"
        },  # this takes what normally goes into ant.configure
        "plot": "boxplot",
        # If you want to add the figure into a subplot...
        #"ax": axs,
    }

    ant = Annotator(None, pairs)
    # We create a FacetGrid and pass the dataframe that we want to use to later apply our functions (plotting the comparisons between groups)
    g1 = sns.FacetGrid(df_compare_groups, aspect=1.5, height=4)
    # map_dataframe accepts a function, which it then applies to the dataframe that is previously passed in the FacetGrid. It also accepts kwargs which
    # are passed inside the function
    g1.map_dataframe(ant.plot_and_annotate_facets, **kwargs)
    #plt.close()

def create_df_max_rand(df_max, df_rand_big, model_type):
    df_max_rand = pd.DataFrame()

    for row in df_max.iterrows():
        ptid = (row[1]["PTID"])
        w = (row[1]["w"])
        b = (row[1]["b"])
        df_max_rand = pd.concat([df_max_rand, df_rand_big[(df_rand_big["PTID"] == ptid) & (df_rand_big["w"] == w) & (df_rand_big["b"] == b)]])
        df_max_rand["Model_type"] = model_type
    return df_max_rand

def plot_save_fig2(df_max_comp_score_a_cn_mmse, df_max_comp_score_a_mci_mmse, df_max_comp_score_G_cn_mmse, df_max_comp_score_G_mci_mmse):
    fig, axs = plt.subplots(nrows = 3, ncols = 2, figsize=(8, 12))

    # Values that give the best composite score
    sns.regplot(
        ax=axs[0, 0],
        data=df_max_comp_score_a_cn_mmse,
        x="wmh_load",
        y="a",
        label="CU WMH",
    )
    sns.regplot(
        ax=axs[0, 0],
        data=df_max_comp_score_a_mci_mmse,
        x="wmh_load",
        y="a",
        label="MCI WMH",
    )

    sns.regplot(
        ax=axs[0, 1],
        data=df_max_comp_score_G_cn_mmse,
        x="wmh_load",
        y="G",
        label="CU WMH",

    )
    sns.regplot(
        ax=axs[0, 1],
        data=df_max_comp_score_G_mci_mmse,
        x="wmh_load",
        y="G",
        label="MCI WMH",

    )

    sns.regplot(x = "a", y = "MMSE", data = df_max_comp_score_a_cn_mmse, label = "CU WMH", ax = axs[1, 0])
    sns.regplot(x = "a", y = "MMSE", data = df_max_comp_score_a_mci_mmse, label = "MCI WMH", ax = axs[1, 0])
    sns.regplot(x = "G", y = "MMSE", data = df_max_comp_score_G_cn_mmse, label = "CU WMH", ax = axs[1, 1])
    sns.regplot(x = "G", y = "MMSE", data = df_max_comp_score_G_mci_mmse, label = "MCI WMH", ax = axs[1, 1])

    sns.regplot(x = "a", y = "PTEDUCAT", data = df_max_comp_score_a_cn_mmse, label = "CU WMH", ax = axs[2, 0])
    sns.regplot(x = "a", y = "PTEDUCAT", data = df_max_comp_score_a_mci_mmse, label = "MCI WMH", ax = axs[2, 0])
    sns.regplot(x = "G", y = "PTEDUCAT", data = df_max_comp_score_G_cn_mmse, label = "CU WMH", ax = axs[2, 1])
    sns.regplot(x = "G", y = "PTEDUCAT", data = df_max_comp_score_G_mci_mmse, label = "MCI WMH", ax = axs[2, 1])

    axs[0, 0].set_xlabel("Normalized WMH volume")
    axs[0, 1].set_xlabel("Normalized WMH volume")
    axs[0, 0].legend()
    axs[0, 1].legend()
    axs[1, 0].legend(loc = "lower left")
    axs[1, 1].legend()
    axs[2, 0].legend()
    axs[2, 1].legend()


    axs = axs.flat
    for n, ax in enumerate(axs):
        ax.text(
            -0.1,
            1.1,
            string.ascii_uppercase[n],
            transform=ax.transAxes,
            size=13,
            weight="bold",
        )
    fig.tight_layout()
    plt.savefig(FIG_DIR / "Figure2.png")
    
    # Linear fit between a, G and WMH load in CN WMH and MCI WMH
    w_a_cn, b_a_cn = np.polyfit(df_max_comp_score_a_cn_mmse["wmh_load"],  df_max_comp_score_a_cn_mmse["a"], 1) 
    w_G_cn, b_G_cn = np.polyfit(df_max_comp_score_G_cn_mmse["wmh_load"],  df_max_comp_score_G_cn_mmse["G"], 1) 
    w_a_mci, b_a_mci = np.polyfit(df_max_comp_score_a_mci_mmse["wmh_load"],  df_max_comp_score_a_mci_mmse["a"], 1) 
    w_G_mci, b_G_mci = np.polyfit(df_max_comp_score_G_mci_mmse["wmh_load"],  df_max_comp_score_G_mci_mmse["G"], 1) 
    
    print("Relationship between a and G and normalized WMH volumes")
    print("#######################################################")
    print(f"In CU WMH: a = -0.02 {round(w_a_cn, 3)} * wmh {round(b_a_cn + 0.02, 2)}")
    print(f"In CU WMH: G = 1.9 {round(w_G_cn, 2)} * wmh {round(b_G_cn - 1.9, 2)}")
    print(f"In MCI WMH: a = -0.02 {round(w_a_mci, 3)} * wmh {round(b_a_mci + 0.02, 2)}")
    print(f"In MCI WMH: G = 1.9 {round(w_G_mci, 2)} * wmh {round(b_G_mci - 1.9, 2)}")

## Prepare data

In [ ]:
# Load wmh dictionary
wmh_dict = get_wmh_load_homogeneous(subjs_to_sim)
# Create overall dfs for each model and populate them with single subject results
df_a_big = pd.DataFrame()
df_a_rand_big = pd.DataFrame()
df_sc_big = pd.DataFrame()
df_G_big = pd.DataFrame()
df_G_rand_big = pd.DataFrame()
df_het_big = pd.DataFrame()
df_het_rand_big = pd.DataFrame()

# We loop through every subject
for subj in subjs_to_sim[:-1]:
    # we read the csv of saved results for each subject for all models
    df_a = pd.read_csv(
        A_DIR / f"sub-{subj}_df_results_homogeneous_a-weight.csv", index_col=0
    )
    df_a_rand = pd.read_csv(
        A_RAND_DIR / f"sub-{subj}_df_results_homogeneous_a-weight.csv",
        index_col=0,
    )
    df_het = pd.read_csv(
        HET_DIR / f"sub-{subj}_df_results_heterogeneous.csv",
        index_col=0,
    )
    df_het_rand = pd.read_csv(
        HET_RAND_DIR / f"sub-{subj}_df_results_heterogeneous.csv",
        index_col=0,
    )
    df_G = pd.read_csv(
        G_DIR / f"sub-{subj}_df_results_homogeneous_G-weight.csv", index_col=0
    )
    df_G_rand = pd.read_csv(
        G_RAND_DIR / f"sub-{subj}_df_results_homogeneous_G-weight.csv", index_col=0
    )
    df_sc = pd.read_csv(SC_DIR / f"sub-{subj}_df_results_disconn.csv", index_col=0)

    # Set the PTID column == to this subject
    df_a["PTID"] = subj
    df_a_rand["PTID"] = subj
    df_het["PTID"] = subj
    df_het_rand["PTID"] = subj
    df_G["PTID"] = subj
    df_G_rand["PTID"] = subj
    df_sc["PTID"] = subj

    # Get the wmh load for this subject
    df_a["wmh_load"] = wmh_dict[subj]
    df_G["wmh_load"] = wmh_dict[subj]

    # we concatenate results of all subjects and create model-specific datasets with all patients
    df_a_big = pd.concat([df_a_big, df_a], ignore_index=True)
    df_a_rand_big = pd.concat([df_a_rand_big, df_a_rand], ignore_index=True)
    df_het_big = pd.concat([df_het_big, df_het], ignore_index=True)
    df_het_rand_big = pd.concat([df_het_rand_big, df_het_rand], ignore_index=True)
    df_G_big = pd.concat([df_G_big, df_G], ignore_index=True)
    df_G_rand_big = pd.concat([df_G_rand_big, df_G_rand], ignore_index=True)
    df_sc_big = pd.concat([df_sc_big, df_sc], ignore_index=True)

# The baseline databases is with a = -0.02 and G = 1.9 == homogeneous a-weight database when b and w = 0!
df_base = df_a_big[(df_a_big["w"] == 0) & (df_a_big["b"] == 0)].copy()
df_base["phfcd_ks"] = 1 - df_base["phfcd_ks"] 
# Then, we convert phfcd to 1-phfcd so to have higher numbers = better fits
df_a_big["phfcd_ks"] = 1 - df_a_big["phfcd_ks"]
df_a_rand_big["phfcd_ks"] = 1 - df_a_rand_big["phfcd_ks"]
df_het_big["phfcd_ks"] = 1 - df_het_big["phfcd_ks"]
df_het_rand_big["phfcd_ks"] = 1 - df_het_rand_big["phfcd_ks"]
df_G_big["phfcd_ks"] = 1 - df_G_big["phfcd_ks"]
df_G_rand_big["phfcd_ks"] = 1 - df_G_rand_big["phfcd_ks"]
df_sc_big["phfcd_ks"] = 1 - df_sc_big["phfcd_ks"]

# Make sure that we didn't mistakenly simulate any positive weights, in case exclude them
df_a_big = df_a_big[(df_a_big["b"] <= 0) & (df_a_big["w"] <= 0)].copy()
df_a_rand_big = df_a_rand_big[(df_a_rand_big["b"] <= 0) & (df_a_rand_big["w"] <= 0)].copy()
df_het_big = df_het_big[(df_het_big["b"] <= 0) & (df_het_big["w"] <= 0)].copy()
df_het_rand_big = df_het_rand_big[(df_het_rand_big["b"] <= 0) & (df_het_rand_big["w"] <= 0)].copy()
df_G_big = df_G_big[(df_G_big["b"] <= 0) & (df_G_big["w"] <= 0)].copy()
df_G_rand_big = df_G_rand_big[(df_G_rand_big["b"] <= 0) & (df_G_rand_big["w"] <= 0)].copy()

# In the formula for the heterogeneous model, I mistakenly performed the simulations with a - w*WMH, so to be consistent with the
# other simulations we need to invert the weights...
# df_het_big["w"] = df_het_big["w"] * -1
# df_het_rand_big["w"] = df_het_rand_big["w"] * -1


# Load the clinical dataframe with the groupings
df_petTOAD_pre = pd.read_csv(RES_DIR / "df_petTOAD.csv")
df_petTOAD_pre["PTID"] = df_petTOAD_pre["PTID"].str.replace("_", "")
df_petTOAD_pre = df_petTOAD_pre.rename(columns={"Group_bin_Fazekas_2.0": "Group_Fazekas_2.0"})
df_petTOAD_pre['WMH_bin'] = np.where(df_petTOAD_pre["Group_Fazekas_2.0"].str.contains("no_WMH"), "no_WMH", "WMH")
df_petTOAD = df_petTOAD_pre[df_petTOAD_pre["WMH_load_subj_space"] < 80000]


# Prepare databases with the maximum values for each type of model
df_max_fc_a, df_max_phfcd_a, df_max_comp_score_a = get_best_dfs(
    df_a_big, "fc_pearson", "phfcd_ks", "comp_score", wmh_dict
)
df_max_fc_a_rand, df_max_phfcd_a_rand, df_max_comp_score_a_rand = get_best_dfs(
    df_a_rand_big, "fc_pearson", "phfcd_ks", "comp_score", wmh_dict
)
df_max_fc_hetero, df_max_phfcd_hetero, df_max_comp_score_hetero = get_best_dfs(
    df_het_big, "fc_pearson", "phfcd_ks", "comp_score", wmh_dict
)
df_max_fc_hetero_rand, df_max_phfcd_hetero_rand, df_max_comp_score_hetero_rand = get_best_dfs(
    df_het_rand_big, "fc_pearson", "phfcd_ks", "comp_score", wmh_dict
)
df_max_fc_G, df_max_phfcd_G, df_max_comp_score_G = get_best_dfs(df_G_big, "fc_pearson", "phfcd_ks", "comp_score", wmh_dict
)
df_max_fc_G_rand, df_max_phfcd_G_rand, df_max_comp_score_G_rand = get_best_dfs(
    df_G_rand_big, "fc_pearson", "phfcd_ks", "comp_score", wmh_dict
)
df_max_fc_sc, df_max_phfcd_sc, df_max_comp_score_sc = get_best_dfs(
    df_sc_big, "fc_pearson", "phfcd_ks", "comp_score", wmh_dict
)

In [ ]:
assert (
    df_a_big["PTID"].unique().shape[0]
    & df_a_rand_big["PTID"].unique().shape[0]
    & df_sc_big["PTID"].unique().shape[0]
    & df_G_big["PTID"].unique().shape[0]
    & df_G_rand_big["PTID"].unique().shape[0]
    & df_het_big["PTID"].unique().shape[0]
    & df_het_rand_big["PTID"].unique().shape[0]
    & df_base["PTID"].unique().shape[0]
) == len(subjs_to_sim[:-1]), "Something went wrong with loading"

### Prepare dfs for boxplots

In [ ]:
# Find the best b and w for the a-weight, G-weight and heterogeneous models
(
    best_fit_b_CN_a,
    best_fit_w_CN_a,
    best_fit_b_mci_a,
    best_fit_w_mci_a,
) = perform_group_analyses(df_petTOAD, df_a_big,  random=False)
(
    best_fit_b_CN_G,
    best_fit_w_CN_G,
    best_fit_b_mci_G,
    best_fit_w_mci_G,
) = perform_group_analyses(df_petTOAD, df_G_big, random=False)
(
    best_fit_b_CN_het,
    best_fit_w_CN_het,
    best_fit_b_mci_het,
    best_fit_w_mci_het,
) = perform_group_analyses(df_petTOAD, df_het_big, False)

In [ ]:
# Set the random best to the same value as the non-random best
best_fit_b_CN_a_rand, best_fit_w_CN_a_rand = best_fit_b_CN_a, best_fit_w_CN_a
best_fit_b_CN_G_rand, best_fit_w_CN_G_rand = best_fit_b_CN_G, best_fit_w_CN_G
best_fit_b_CN_het_rand, best_fit_w_CN_het_rand = best_fit_b_CN_het, best_fit_w_CN_het


# We create merged databases with clinical information for the remaining models
df_a_big_boxplot = pd.merge(
    df_a_big, df_petTOAD[["PTID", "Group_Fazekas_2.0"]], on="PTID"
)

df_a_rand_big_boxplot = pd.merge(
    df_a_rand_big, df_petTOAD[["PTID", "Group_Fazekas_2.0"]], on="PTID"
)

df_het_big_boxplot = pd.merge(
    df_het_big, df_petTOAD[["PTID", "Group_Fazekas_2.0"]], on="PTID"
)

df_het_rand_big_boxplot = pd.merge(
    df_het_rand_big, df_petTOAD[["PTID", "Group_Fazekas_2.0"]], on="PTID"
)

df_G_big_boxplot = pd.merge(
    df_G_big, df_petTOAD[["PTID", "Group_Fazekas_2.0"]], on="PTID"
)

df_G_rand_big_boxplot = pd.merge(
    df_G_rand_big, df_petTOAD[["PTID", "Group_Fazekas_2.0"]], on="PTID"
)

df_sc_big_boxplot = pd.merge(
    df_sc_big, df_petTOAD[["PTID", "Group_Fazekas_2.0"]], on="PTID"
)

# Create df with only the best values from before for CN
df_boxplot_compare_CN_base = create_df_boxplot(df_a_big_boxplot, 0, 0, "CN_WMH", "Base")
df_boxplot_compare_CN_a = create_df_boxplot(
    df_a_big_boxplot, best_fit_b_CN_a, best_fit_w_CN_a, "CN_WMH", "Homo."
)
df_boxplot_compare_CN_a_rand = create_df_boxplot(
    df_a_rand_big_boxplot,
    best_fit_b_CN_a_rand,
    best_fit_w_CN_a_rand,
    "CN_WMH",
    "Homo. rand.",
)
df_boxplot_compare_CN_het = create_df_boxplot(
    df_het_big_boxplot, best_fit_b_CN_het, best_fit_w_CN_het, "CN_WMH", "Hetero."
)
df_boxplot_compare_CN_het_rand = create_df_boxplot(
    df_het_rand_big_boxplot,
    best_fit_b_CN_het_rand,
    best_fit_w_CN_het_rand,
    "CN_WMH",
    "Hetero. rand.",
)
df_boxplot_compare_CN_G = create_df_boxplot(
    df_G_big_boxplot, best_fit_b_CN_G, best_fit_w_CN_G, "CN_WMH", "G-weight"
)
df_boxplot_compare_CN_G_rand = create_df_boxplot(
    df_G_rand_big_boxplot,
    best_fit_b_CN_G_rand,
    best_fit_w_CN_G_rand,
    "CN_WMH",
    "G rand.",
)
df_boxplot_compare_CN_sc = df_sc_big_boxplot[
    df_sc_big_boxplot["Group_Fazekas_2.0"] == "CN_WMH"
].drop(columns="Group_Fazekas_2.0")
# The structural disconnectivity model doesn't have a weight, so its processing is slightly different
df_boxplot_compare_CN_sc["comp_score"] = (
    df_boxplot_compare_CN_sc["fc_pearson"] + 1 * df_boxplot_compare_CN_sc["phfcd_ks"]
)
df_boxplot_compare_CN_sc["Model_type"] = "Disconn."

df_boxplot_compare_CN = pd.concat(
    [
        df_boxplot_compare_CN_base,
        df_boxplot_compare_CN_a,
        df_boxplot_compare_CN_het,
        df_boxplot_compare_CN_G,
        df_boxplot_compare_CN_sc,
    ]
)

In [ ]:
# Set the random best to the same value as the non-random best
best_fit_b_mci_a_rand, best_fit_w_mci_a_rand = best_fit_b_mci_a, best_fit_w_mci_a
best_fit_b_mci_G_rand, best_fit_w_mci_G_rand = best_fit_b_mci_G, best_fit_w_mci_G
best_fit_b_mci_het_rand, best_fit_w_mci_het_rand = (
    best_fit_b_mci_het,
    best_fit_w_mci_het,
)

# Perform the same steps done with CN but on MCI
# Create df with only the best values from before for MCI
df_boxplot_compare_mci_base = create_df_boxplot(
    df_a_big_boxplot, 0, 0, "MCI_WMH", "Base"
)
df_boxplot_compare_mci_a = create_df_boxplot(
    df_a_big_boxplot, best_fit_b_mci_a, best_fit_w_mci_a, "MCI_WMH", "Homo."
)
df_boxplot_compare_mci_a_rand = create_df_boxplot(
    df_a_rand_big_boxplot,
    best_fit_b_mci_a_rand,
    best_fit_w_mci_a_rand,
    "MCI_WMH",
    "Homo. rand.",
)
df_boxplot_compare_mci_het = create_df_boxplot(
    df_het_big_boxplot, best_fit_b_mci_het, best_fit_w_mci_het, "MCI_WMH", "Hetero."
)
df_boxplot_compare_mci_het_rand = create_df_boxplot(
    df_het_rand_big_boxplot,
    best_fit_b_mci_het_rand,
    best_fit_w_mci_het_rand,
    "MCI_WMH",
    "Hetero. rand.",
)
df_boxplot_compare_mci_G = create_df_boxplot(
    df_G_big_boxplot, best_fit_b_mci_G, best_fit_w_mci_G, "MCI_WMH", "G-weight"
)
df_boxplot_compare_mci_G_rand = create_df_boxplot(
    df_G_rand_big_boxplot,
    best_fit_b_mci_G_rand,
    best_fit_w_mci_G_rand,
    "MCI_WMH",
    "G rand.",
)
df_boxplot_compare_mci_sc = df_sc_big_boxplot[
    df_sc_big_boxplot["Group_Fazekas_2.0"] == "MCI_WMH"
].drop(columns="Group_Fazekas_2.0")
df_boxplot_compare_mci_sc["comp_score"] = (
    df_boxplot_compare_mci_sc["fc_pearson"] + 1 * df_boxplot_compare_mci_sc["phfcd_ks"]
)
df_boxplot_compare_mci_sc["Model_type"] = "Disconn."
df_boxplot_compare_mci = pd.concat(
    [
        df_boxplot_compare_mci_base,
        df_boxplot_compare_mci_a,
        df_boxplot_compare_mci_het,
        df_boxplot_compare_mci_G,
        df_boxplot_compare_mci_sc,
    ]
)

In [ ]:
# Create a unified long df to compare the model performance between random vs. non-random
df_boxplot_compare_rand_cn = pd.concat(
    [
        df_boxplot_compare_CN_a,
        df_boxplot_compare_CN_a_rand,
        df_boxplot_compare_CN_het,
        df_boxplot_compare_CN_het_rand,
        df_boxplot_compare_CN_G,
        df_boxplot_compare_CN_G_rand,
    ]
)
df_boxplot_compare_rand_cn["Random"] = np.where(
    df_boxplot_compare_rand_cn["Model_type"].str.contains("rand."),
    "Random",
    "Not Random",
)
df_boxplot_compare_rand_mci = pd.concat(
    [
        df_boxplot_compare_mci_a,
        df_boxplot_compare_mci_a_rand,
        df_boxplot_compare_mci_het,
        df_boxplot_compare_mci_het_rand,
        df_boxplot_compare_mci_G,
        df_boxplot_compare_mci_G_rand,
    ]
)
df_boxplot_compare_rand_mci["Random"] = np.where(
    df_boxplot_compare_rand_mci["Model_type"].str.contains("rand."),
    "Random",
    "Not Random",
)
# Create a unified long df to compare the model performance between groups
df_boxplot_compare_CN["group"] = "CN WMH"
df_boxplot_compare_mci["group"] = "MCI WMH"
df_boxplot_compare = pd.concat([df_boxplot_compare_CN, df_boxplot_compare_mci])

### Prepare dfs with maximum values achieved

In [ ]:
# Now let's plot the best FC and 1-phFCDs achievable in the range of explored parameters
df_base_merged = pd.merge(df_base, df_petTOAD[["PTID", "Group_Fazekas_2.0"]], on="PTID")
df_base_merged["comp_score"] = df_base_merged["fc_pearson"] + 1 *  df_base_merged["phfcd_ks"]
df_max_comp_score_a_merged = pd.merge(
    df_max_comp_score_a, df_petTOAD[["PTID", "Group_Fazekas_2.0"]], on="PTID"
)
df_max_comp_score_a_rand_merged = pd.merge(
    df_max_comp_score_a_rand, df_petTOAD[["PTID", "Group_Fazekas_2.0"]], on="PTID"
)
df_max_comp_score_hetero_merged = pd.merge(
    df_max_comp_score_hetero, df_petTOAD[["PTID", "Group_Fazekas_2.0"]], on="PTID"
)
df_max_comp_score_G_merged = pd.merge(
    df_max_comp_score_G, df_petTOAD[["PTID", "Group_Fazekas_2.0"]], on="PTID"
)
df_max_comp_score_G_rand_merged = pd.merge(
    df_max_comp_score_G_rand, df_petTOAD[["PTID", "Group_Fazekas_2.0"]], on="PTID"
)
df_max_comp_score_sc_merged = pd.merge(
    df_max_comp_score_sc, df_petTOAD[["PTID", "Group_Fazekas_2.0"]], on="PTID"
)

df_base_merged["Model_type"] = "Base"

df_max_comp_score_a_merged["Model_type"] = "Homo."
df_max_comp_score_a_rand_merged["Model_type"] = "Homo. rand."
df_max_comp_score_hetero_merged["Model_type"] = "Hetero."
df_max_comp_score_G_merged["Model_type"] = "G-weight"
df_max_comp_score_G_rand_merged["Model_type"] = "G rand."
df_max_comp_score_sc_merged["Model_type"] = "Disconn."

# df_max_phfcd_a_merged = pd.merge(
#     df_max_phfcd_a, df_petTOAD[["PTID", "Group_Fazekas_2.0"]], on="PTID"
# )
# df_max_phfcd_a_rand_merged = pd.merge(
#     df_max_phfcd_a_rand, df_petTOAD[["PTID", "Group_Fazekas_2.0"]], on="PTID"
# )
# df_max_phfcd_sc_merged = pd.merge(
#     df_max_phfcd_sc, df_petTOAD[["PTID", "Group_Fazekas_2.0"]], on="PTID"
# )
# df_max_phfcd_G_merged = pd.merge(
#     df_max_phfcd_G, df_petTOAD[["PTID", "Group_Fazekas_2.0"]], on="PTID"
# )
# df_max_phfcd_G_rand_merged = pd.merge(
#     df_max_phfcd_G_rand, df_petTOAD[["PTID", "Group_Fazekas_2.0"]], on="PTID"
# )
# df_max_phfcd_hetero_merged = pd.merge(
#     df_max_phfcd_hetero, df_petTOAD[["PTID", "Group_Fazekas_2.0"]], on="PTID"
# )

# df_max_phfcd_a_merged["Model_type"] = "Homo."
# df_max_phfcd_a_rand_merged["Model_type"] = "Homo. rand."
# df_max_phfcd_hetero_merged["Model_type"] = "Hetero."
# df_max_phfcd_G_merged["Model_type"] = "G-weight"
# df_max_phfcd_G_rand_merged["Model_type"] = "G rand."
# df_max_phfcd_sc_merged["Model_type"] = "Disconn."

In [ ]:
# Select only CN
df_base_merged_CN = df_base_merged[df_base_merged["Group_Fazekas_2.0"] == "CN_WMH"]
df_max_comp_score_a_merged_CN = df_max_comp_score_a_merged[
    df_max_comp_score_a_merged["Group_Fazekas_2.0"] == "CN_WMH"
]
df_max_comp_score_a_rand_merged_CN = df_max_comp_score_a_rand_merged[
    df_max_comp_score_a_rand_merged["Group_Fazekas_2.0"] == "CN_WMH"
]
df_max_comp_score_hetero_merged_CN = df_max_comp_score_hetero_merged[
    df_max_comp_score_hetero_merged["Group_Fazekas_2.0"] == "CN_WMH"
]
df_max_comp_score_G_merged_CN = df_max_comp_score_G_merged[
    df_max_comp_score_G_merged["Group_Fazekas_2.0"] == "CN_WMH"
]
df_max_comp_score_G_rand_merged_CN = df_max_comp_score_G_rand_merged[
    df_max_comp_score_G_rand_merged["Group_Fazekas_2.0"] == "CN_WMH"
]
df_max_comp_score_sc_merged_CN = df_max_comp_score_sc_merged[
    df_max_comp_score_sc_merged["Group_Fazekas_2.0"] == "CN_WMH"
]

# df_max_phfcd_a_merged_CN = df_max_phfcd_a_merged[
#     df_max_phfcd_a_merged["Group_Fazekas_2.0"] == "CN_WMH"
# ]
# df_max_phfcd_a_rand_merged_CN = df_max_phfcd_a_rand_merged[
#     df_max_phfcd_a_rand_merged["Group_Fazekas_2.0"] == "CN_WMH"
# ]
# df_max_phfcd_hetero_merged_CN = df_max_phfcd_hetero_merged[
#     df_max_phfcd_hetero_merged["Group_Fazekas_2.0"] == "CN_WMH"
# ]
# df_max_phfcd_hetero_merged_CN = df_max_phfcd_hetero_merged_CN.dropna()
# df_max_phfcd_G_merged_CN = df_max_phfcd_G_merged[
#     df_max_phfcd_G_merged["Group_Fazekas_2.0"] == "CN_WMH"
# ]
# df_max_phfcd_G_rand_merged_CN = df_max_phfcd_G_rand_merged[
#     df_max_phfcd_G_rand_merged["Group_Fazekas_2.0"] == "CN_WMH"
# ]
# df_max_phfcd_sc_merged_CN = df_max_phfcd_sc_merged[
#     df_max_phfcd_sc_merged["Group_Fazekas_2.0"] == "CN_WMH"
# ]

df_max_comp_score_CN = pd.concat(
    [
        df_base_merged_CN,
        df_max_comp_score_a_merged_CN,
        df_max_comp_score_hetero_merged_CN,
        df_max_comp_score_G_merged_CN,
        df_max_comp_score_sc_merged_CN,
    ]
)

# df_max_phfcd_CN = pd.concat(
#     [
#         df_base_merged_CN,
#         df_max_phfcd_a_merged_CN,
#         df_max_phfcd_hetero_merged_CN,
#         df_max_phfcd_G_merged_CN,
#         df_max_phfcd_sc_merged_CN,
#     ]
# )

In [ ]:
# Select only MCI
df_base_merged_mci = df_base_merged[df_base_merged["Group_Fazekas_2.0"] == "MCI_WMH"]
df_max_comp_score_a_merged_mci = df_max_comp_score_a_merged[
    df_max_comp_score_a_merged["Group_Fazekas_2.0"] == "MCI_WMH"
]
df_max_comp_score_a_rand_merged_mci = df_max_comp_score_a_rand_merged[
    df_max_comp_score_a_rand_merged["Group_Fazekas_2.0"] == "MCI_WMH"
]
df_max_comp_score_hetero_merged_mci = df_max_comp_score_hetero_merged[
    df_max_comp_score_hetero_merged["Group_Fazekas_2.0"] == "MCI_WMH"
]
df_max_comp_score_G_merged_mci = df_max_comp_score_G_merged[
    df_max_comp_score_G_merged["Group_Fazekas_2.0"] == "MCI_WMH"
]
df_max_comp_score_G_rand_merged_mci = df_max_comp_score_G_rand_merged[
    df_max_comp_score_G_rand_merged["Group_Fazekas_2.0"] == "MCI_WMH"
]
df_max_comp_score_sc_merged_mci = df_max_comp_score_sc_merged[
    df_max_comp_score_sc_merged["Group_Fazekas_2.0"] == "MCI_WMH"
]

# df_max_phfcd_a_merged_mci = df_max_phfcd_a_merged[
#     df_max_phfcd_a_merged["Group_Fazekas_2.0"] == "MCI_WMH"
# ]
# df_max_phfcd_a_rand_merged_mci = df_max_phfcd_a_rand_merged[
#     df_max_phfcd_a_rand_merged["Group_Fazekas_2.0"] == "MCI_WMH"
# ]
# # This dataset contain multiple values for one pts, so we filter them out..
# df_max_phfcd_a_rand_merged_mci = df_max_phfcd_a_rand_merged_mci.drop_duplicates(
#     ["PTID"]
# )
# df_max_phfcd_hetero_merged_mci = df_max_phfcd_hetero_merged[
#     df_max_phfcd_hetero_merged["Group_Fazekas_2.0"] == "MCI_WMH"
# ]
# df_max_phfcd_G_merged_mci = df_max_phfcd_G_merged[
#     df_max_phfcd_G_merged["Group_Fazekas_2.0"] == "MCI_WMH"
# ]
# df_max_phfcd_G_rand_merged_mci = df_max_phfcd_G_rand_merged[
#     df_max_phfcd_G_rand_merged["Group_Fazekas_2.0"] == "MCI_WMH"
# ]
# df_max_phfcd_sc_merged_mci = df_max_phfcd_sc_merged[
#     df_max_phfcd_sc_merged["Group_Fazekas_2.0"] == "MCI_WMH"
# ]

df_max_comp_score_mci = pd.concat(
    [
        df_base_merged_mci,
        df_max_comp_score_a_merged_mci,
        df_max_comp_score_hetero_merged_mci,
        df_max_comp_score_G_merged_mci,
        df_max_comp_score_sc_merged_mci,
    ]
)
# df_max_phfcd_mci = pd.concat(
#     [
#         df_base_merged_mci,
#         df_max_phfcd_a_merged_mci,
#         df_max_phfcd_hetero_merged_mci,
#         df_max_phfcd_G_merged_mci,
#         df_max_phfcd_sc_merged_mci,
#     ]
# )

In [ ]:
df_max_comp_score_a["a"] = (
    -0.02
    + df_max_comp_score_a["w"] * df_max_comp_score_a["wmh_load"]
    + df_max_comp_score_a["b"]
)
df_max_comp_score_G["G"] = (
    1.9
    + df_max_comp_score_G["w"] * df_max_comp_score_G["wmh_load"]
    + df_max_comp_score_G["b"]
)

df_max_comp_score_a_cn = df_max_comp_score_a[df_max_comp_score_a["PTID"].isin(CN_WMH)]
df_max_comp_score_a_mci = df_max_comp_score_a[df_max_comp_score_a["PTID"].isin(MCI_WMH)]
df_max_comp_score_G_cn = df_max_comp_score_G[df_max_comp_score_G["PTID"].isin(CN_WMH)]
df_max_comp_score_G_mci = df_max_comp_score_G[df_max_comp_score_G["PTID"].isin(MCI_WMH)]

df_max_comp_score_a_mci_mmse = pd.merge(df_max_comp_score_a_mci, df_petTOAD, on = "PTID")
df_max_comp_score_a_cn_mmse = pd.merge(df_max_comp_score_a_cn, df_petTOAD, on = "PTID")
df_max_comp_score_G_mci_mmse = pd.merge(df_max_comp_score_G_mci, df_petTOAD, on = "PTID")
df_max_comp_score_G_cn_mmse = pd.merge(df_max_comp_score_G_cn, df_petTOAD, on = "PTID")

In [ ]:
df_max_comp_score_CN["group"] = "CN WMH"
df_max_comp_score_mci["group"] = "MCI WMH"
df_max_boxplot_compare = pd.concat([df_max_comp_score_CN, df_max_comp_score_mci])
# create tuples
df_max_rand_compare = pd.DataFrame()
list_tuples_df = [(df_max_comp_score_a, df_a_rand_big, "Homo."), (df_max_comp_score_G, df_G_rand_big, "G-weight"), (df_max_comp_score_hetero, df_het_rand_big, "Hetero.")]
for tp in list_tuples_df:
    df_max_rand_compare= pd.concat([df_max_rand_compare, create_df_max_rand(tp[0], tp[1], tp[2])])
df_max_rand_compare["Group_Fazekas_2.0"] = np.where(df_max_rand_compare["PTID"].isin(CN_WMH), "CN_WMH", "MCI_WMH")
df_max_rand_compare["Random"] = "Random"
df_max_rand_compare["Model_type_composite"] = df_max_rand_compare["Model_type"] + " " + "Random"
df_max_rand_compare_cn = df_max_rand_compare[df_max_rand_compare["Group_Fazekas_2.0"] == "CN_WMH"]
df_max_rand_compare_mci = df_max_rand_compare[df_max_rand_compare["Group_Fazekas_2.0"] == "MCI_WMH"]
df_max_comp_score_CN["Random"] = "Not Random"
df_max_comp_score_mci["Random"] = "Not Random"
df_max_comp_score_CN["Model_type_composite"] = df_max_comp_score_CN["Model_type"] + df_max_comp_score_CN["Random"] 
df_max_comp_score_mci["Model_type_composite"] = df_max_comp_score_mci["Model_type"] + df_max_comp_score_mci["Random"] 

df_max_comp_score_boxplot_compare_rand_cn = pd.concat([df_max_rand_compare_cn, df_max_comp_score_CN])
df_max_comp_score_boxplot_compare_rand_mci = pd.concat([df_max_rand_compare_mci, df_max_comp_score_mci])
df_max_comp_score_boxplot_compare_rand_cn = df_max_comp_score_boxplot_compare_rand_cn[(df_max_comp_score_boxplot_compare_rand_cn["Model_type"] != "Base") & (df_max_comp_score_boxplot_compare_rand_cn["Model_type"] != "Disconn.")]
df_max_comp_score_boxplot_compare_rand_mci = df_max_comp_score_boxplot_compare_rand_mci[(df_max_comp_score_boxplot_compare_rand_mci["Model_type"] != "Base") & (df_max_comp_score_boxplot_compare_rand_mci["Model_type"] != "Disconn.")]

# Analyses

## Summary statistics and tables

In [ ]:
#Read in data
n_pts = len(df_petTOAD)
age_min = round(df_petTOAD.describe()["Age"]["min"], 0)
age_max = round(df_petTOAD.describe()["Age"]["max"],0)
sex_counts = df_petTOAD['Sex'].value_counts()
DX_counts = df_petTOAD['Group'].value_counts()
females_num = sex_counts["F"]
males_num = sex_counts["M"]
cu_num = DX_counts['CN']
mci_num = DX_counts['MCI']

cu_no_wmh = len(df_petTOAD[df_petTOAD['Group_Fazekas_2.0'] == 'CN_no_WMH'])
cu_wmh = len(df_petTOAD[df_petTOAD['Group_Fazekas_2.0'] == 'CN_WMH'])
mci_no_wmh = len(df_petTOAD[df_petTOAD['Group_Fazekas_2.0'] == 'MCI_no_WMH'])
mci_wmh = len(df_petTOAD[df_petTOAD['Group_Fazekas_2.0'] == 'MCI_WMH'])

print(f"After exclusion, {n_pts} patients (age-range: {age_min}-{age_max}, {females_num} ({round(females_num / n_pts * 100, 2)}%) females, {males_num} ({round(males_num / n_pts * 100, 2)}%) males, {cu_num} ({round(cu_num / n_pts* 100, 1)}%) CU and {mci_num} ({round(mci_num / n_pts * 100, 1)}%) MCI) were considered for the modeling pipeline.")
print(f"Patients were further subdivided into subgroups according to the previously defined Fazekas score cutoff of <= 2.")
print(f"Accordingly, there were: {cu_no_wmh} ({round(cu_no_wmh / cu_num * 100, 1)}%) CU without WMH, {cu_wmh} ({round(cu_wmh / cu_num * 100, 1)}%) CU with WMH, {mci_no_wmh} ({round(mci_no_wmh / mci_num * 100, 1)}%) MCI without WMH, {mci_wmh} ({round(mci_wmh / mci_num* 100, 1)}%) MCI with WMH,")
# Create contingency table for comparing Fazekas binary classification between groups with Chi-squared
ct_wmh = pd.crosstab(df_petTOAD['WMH_bin'], df_petTOAD['Group'])
_, p_wmh, _, _ = stats.chi2_contingency(ct_wmh)
if p_wmh < 0.05:
    print(f"with statistically significant differences in frequency between subgroups (p = {round(p_wmh,3)})")
else:
    print(f"with no statistically significant differences in frequency between subgroups (p = {round(p_wmh, 3)})")
# Get age summary stats
age_summary = df_petTOAD.groupby(["Group"])['Age'].describe()
age_summary_subgroups = df_petTOAD.groupby(["Group_Fazekas_2.0"])['Age'].describe()
print("################# Results #################")
print("# Age #")
pval_age_diff_cn_vs_mci = stats.mannwhitneyu(df_petTOAD[df_petTOAD["Group"] == "CN"]["Age"], df_petTOAD[df_petTOAD["Group"] == "MCI"]["Age"])[1]
pval_age_diff_cn_no_wmh_vs_wmh = stats.mannwhitneyu(df_petTOAD[df_petTOAD["Group_Fazekas_2.0"] == "CN_WMH"]["Age"], df_petTOAD[df_petTOAD["Group_Fazekas_2.0"] == "CN_no_WMH"]["Age"])[1]
pval_age_diff_mci_no_wmh_vs_wmh = stats.mannwhitneyu(df_petTOAD[df_petTOAD["Group_Fazekas_2.0"] == "MCI_WMH"]["Age"], df_petTOAD[df_petTOAD["Group_Fazekas_2.0"] == "MCI_no_WMH"]["Age"])[1]
print(f"Median age for CU subjects = {age_summary['50%']['CN']} (IQR = {age_summary['25%']['CN']} - {age_summary['75%']['CN']})")
print(f"Median age for MCI subjects = {age_summary['50%']['MCI']} (IQR = {age_summary['25%']['MCI']} - {age_summary['75%']['MCI']})")
print(f"P-value age CU vs MCI: {round(pval_age_diff_cn_vs_mci, 5)}")

print(f"Median age for CU no WMH subjects = {age_summary_subgroups['50%']['CN_no_WMH']} (IQR = {age_summary_subgroups['25%']['CN_no_WMH']} - {age_summary_subgroups['75%']['CN_no_WMH']})")
print(f"Median age for CU WMH subjects = {age_summary_subgroups['50%']['CN_WMH']} (IQR = {age_summary_subgroups['25%']['CN_WMH']} - {age_summary_subgroups['75%']['CN_WMH']})")
print(f"P-value age CU no WMH vs CU WMH: {round(pval_age_diff_cn_no_wmh_vs_wmh, 5)}")

print(f"Median age for MCI no WMH subjects = {age_summary_subgroups['50%']['MCI_no_WMH']} (IQR = {age_summary_subgroups['25%']['MCI_no_WMH']} - {age_summary_subgroups['75%']['MCI_no_WMH']})")
print(f"Median age for MCI WMH subjects = {age_summary_subgroups['50%']['MCI_WMH']} (IQR = {age_summary_subgroups['25%']['MCI_WMH']} - {age_summary_subgroups['75%']['MCI_WMH']})")
print(f"P-value age MCI no WMH vs MCI WMH: {round(pval_age_diff_mci_no_wmh_vs_wmh, 5)}")


print("# WMH #")
wmh_summary = df_petTOAD.groupby(["Group"])['WMH_load_subj_space'].describe()
wmh_summary_subgroups = df_petTOAD.groupby(["Group_Fazekas_2.0"])['WMH_load_subj_space'].describe()

pval_wmh_diff_cn_vs_mci= stats.mannwhitneyu(df_petTOAD[df_petTOAD["Group"] == "CN"]["WMH_load_subj_space"], df_petTOAD[df_petTOAD["Group"] == "MCI"]["WMH_load_subj_space"])[1]
pval_wmh_diff_cn_no_wmh_vs_wmh = stats.mannwhitneyu(df_petTOAD[df_petTOAD["Group_Fazekas_2.0"] == "CN_WMH"]["WMH_load_subj_space"], df_petTOAD[df_petTOAD["Group_Fazekas_2.0"] == "CN_no_WMH"]["WMH_load_subj_space"])[1]
pval_wmh_diff_mci_no_wmh_vs_wmh = stats.mannwhitneyu(df_petTOAD[df_petTOAD["Group_Fazekas_2.0"] == "MCI_WMH"]["WMH_load_subj_space"], df_petTOAD[df_petTOAD["Group_Fazekas_2.0"] == "MCI_no_WMH"]["WMH_load_subj_space"])[1]

print(f"Median WMH volume load for CN subjects = {wmh_summary['50%']['CN']} (IQR = {wmh_summary['25%']['CN']} - {wmh_summary['75%']['CN']})")
print(f"Median WMH volume load for MCI subjects = {wmh_summary['50%']['MCI']} (IQR = {wmh_summary['25%']['MCI']} - {wmh_summary['75%']['MCI']})")
print(f"P-value WMH volum CU vs MCI: {pval_wmh_diff_cn_vs_mci}")

print(f"Median WMH volume load for CN no WMH subjects = {wmh_summary_subgroups['50%']['CN_no_WMH']} (IQR = {wmh_summary_subgroups['25%']['CN_no_WMH']} - {wmh_summary_subgroups['75%']['CN_no_WMH']})")
print(f"Median WMH volume load for CN WMH subjects = {wmh_summary_subgroups['50%']['CN_WMH']} (IQR = {wmh_summary_subgroups['25%']['CN_WMH']} - {wmh_summary_subgroups['75%']['CN_WMH']})")
print(f"P-value WMH volume CU no WMH vs CU WMH: {pval_wmh_diff_cn_no_wmh_vs_wmh}")

print(f"Median WMH volume load for MCI no WMH subjects = {wmh_summary_subgroups['50%']['MCI_no_WMH']} (IQR = {wmh_summary_subgroups['25%']['MCI_no_WMH']} - {wmh_summary_subgroups['75%']['MCI_no_WMH']})")
print(f"Median WMH volume load for MCI WMH subjects = {wmh_summary_subgroups['50%']['MCI_WMH']} (IQR = {wmh_summary_subgroups['25%']['MCI_WMH']} - {wmh_summary_subgroups['75%']['MCI_WMH']})")
print(f"P-value WMH volume MCI no WMH vs MCI WMH: {pval_wmh_diff_mci_no_wmh_vs_wmh}")

print("# Sex #")
# Create contingency table for chi-squared
ct_sex = pd.crosstab(df_petTOAD['Sex'], df_petTOAD['Group'])
c_sex, p_sex, _, _ = stats.chi2_contingency(ct_sex)
if p_sex < 0.05:
    print(f"MCI and CU showed statistically significant sex differences, p = {round(p_sex, 3)}")
else:
    print(f"MCI and CU showed no sex groups differences, p = {round(p_sex,3)}")

# Create contingency table for chi-squared
ct_sex_wmh = pd.crosstab(df_petTOAD['Sex'], df_petTOAD['Group_Fazekas_2.0'])
ct_sex_cn = ct_sex_wmh.iloc[:2,:2]
c_sex_cn, p_sex_cn, _, _ = stats.chi2_contingency(ct_sex_cn)
if p_sex_cn < 0.05:
    print(f"CU WMH and CU no WMH showed statistically significant sex differences, p = {round(p_sex_cn, 3)}")
else:
    print(f"CU WMH and CU no WMH showed no sex groups differences, p = {round(p_sex_cn,3)}")

# Create contingency table for chi-squared
ct_sex_mci = ct_sex_wmh.iloc[:2,2:]
c_sex_mci, p_sex_mci, _, _ = stats.chi2_contingency(ct_sex_mci)
if p_sex_mci < 0.05:
    print(f"CU WMH and CU no WMH showed statistically significant sex differences, p = {round(p_sex_mci, 3)}")
else:
    print(f"MCI WMH and MCI no WMH showed no sex groups differences, p = {round(p_sex_mci,3)}")
    

In [ ]:
# Create a summary DataFrame
summary_data = {
    "Age Median (IQR)": [
        f"{age_summary_subgroups['50%']['CN_no_WMH']} ({age_summary_subgroups['25%']['CN_no_WMH']} - {age_summary_subgroups['75%']['CN_no_WMH']})",
        f"{age_summary_subgroups['50%']['CN_WMH']} ({age_summary_subgroups['25%']['CN_WMH']} - {age_summary_subgroups['75%']['CN_WMH']})",
        f"{round(pval_age_diff_cn_no_wmh_vs_wmh, 3)}",
        f"{age_summary_subgroups['50%']['MCI_no_WMH']} ({age_summary_subgroups['25%']['MCI_no_WMH']} - {age_summary_subgroups['75%']['MCI_no_WMH']})",
        f"{age_summary_subgroups['50%']['MCI_WMH']} ({age_summary_subgroups['25%']['MCI_WMH']} - {age_summary_subgroups['75%']['MCI_WMH']})",
        f"{round(pval_age_diff_mci_no_wmh_vs_wmh, 3)}",
    ],
    "Sex n (%)": ["", "", round(p_sex_cn, 3), "", "", round(p_sex_mci)],
    "Woman": [
        f"{ct_sex_cn['CN_no_WMH']['F']} ({round(ct_sex_cn['CN_no_WMH']['F'] / ct_sex_cn['CN_no_WMH'].sum() * 100, 1)}%)",
        f"{ct_sex_cn['CN_WMH']['F']} ({round(ct_sex_cn['CN_WMH']['F'] / ct_sex_cn['CN_WMH'].sum() * 100, 1)}%)",
        "",
        f"{ct_sex_mci['MCI_no_WMH']['F']} ({round(ct_sex_mci['MCI_no_WMH']['F'] / ct_sex_mci['MCI_no_WMH'].sum() * 100, 1)}%)",
        f"{ct_sex_mci['MCI_WMH']['F']} ({round(ct_sex_mci['MCI_WMH']['F'] / ct_sex_mci['MCI_WMH'].sum() * 100, 1)}%)",
        ""
    ],
    "Man": [
        f"{ct_sex_cn['CN_no_WMH']['M']} ({round(ct_sex_cn['CN_no_WMH']['M'] / ct_sex_cn['CN_no_WMH'].sum() * 100, 1)}%)",
        f"{ct_sex_cn['CN_WMH']['M']} ({round(ct_sex_cn['CN_WMH']['M'] / ct_sex_cn['CN_WMH'].sum() * 100, 1)}%)",
        "",
        f"{ct_sex_mci['MCI_no_WMH']['M']} ({round(ct_sex_mci['MCI_no_WMH']['M'] / ct_sex_mci['MCI_no_WMH'].sum() * 100, 1)}%)",
        f"{ct_sex_mci['MCI_WMH']['M']} ({round(ct_sex_mci['MCI_WMH']['M'] / ct_sex_mci['MCI_WMH'].sum() * 100, 1)}%)",
        ""
    ],
    "WMH volume (mm^3)": [
        f"{int(wmh_summary_subgroups['50%']['CN_no_WMH'])} ({int(wmh_summary_subgroups['25%']['CN_no_WMH'])}-{int(wmh_summary_subgroups['75%']['CN_no_WMH'])})",
        f"{int(wmh_summary_subgroups['50%']['CN_WMH'])} ({int(wmh_summary_subgroups['25%']['CN_WMH'])}-{int(wmh_summary_subgroups['75%']['CN_WMH'])})",
        f"{round(pval_wmh_diff_cn_no_wmh_vs_wmh, 5)}",
        f"{int(wmh_summary_subgroups['50%']['MCI_no_WMH'])} ({int(wmh_summary_subgroups['25%']['MCI_no_WMH'])}-{int(wmh_summary_subgroups['75%']['MCI_no_WMH'])})",
        f"{int(wmh_summary_subgroups['50%']['MCI_WMH'])} ({int(wmh_summary_subgroups['25%']['MCI_WMH'])}-{int(wmh_summary_subgroups['75%']['MCI_WMH'])})",
        f"{round(pval_wmh_diff_mci_no_wmh_vs_wmh, 5)}",
    ],
}

# Create the summary DataFrame
df_summary = pd.DataFrame(data=summary_data)

# Print the summary DataFrame
df_table1 = df_summary.T
df_table1.columns = [f"CU no WMH (n = {cu_no_wmh})", f"CU WMH (n = {cu_wmh})",  "p", f"MCI no WMH (n = {mci_no_wmh})", f"MCI WMH (n = {mci_wmh})", "p"]
df_table1.to_csv(RES_DIR / "table_1.csv")
df_table1

In [ ]:
# Create a summary DataFrame
summary_data = {
    "Age Median (IQR)": [
        f"{age_summary['50%']['CN']} ({age_summary['25%']['CN']} - {age_summary['75%']['CN']})",
        f"{age_summary['50%']['MCI']} ({age_summary['25%']['MCI']} - {age_summary['75%']['MCI']})",
        f"{round(pval_age_diff_cn_vs_mci, 4)}"], 
    "Sex n (%)": ["", "", round(p_sex, 3)],
    "Woman": [f"{ct_sex.iloc[0, 0]} ({round(ct_sex.iloc[0, 0]/cu_num * 100, 1)}%)", f"{ct_sex.iloc[0, 1]} ({round(ct_sex.iloc[0, 1] / mci_num * 100, 1)}%)", ""],
    "Man": [f"{ct_sex.iloc[1, 0]} ({round(ct_sex.iloc[1, 0]/cu_num * 100, 1)}%)", f"{ct_sex.iloc[1, 1]} ({round(ct_sex.iloc[1, 1] / mci_num * 100, 1)}%)", ""],
    "WMH volume (mm^3)": [
        f"{int(wmh_summary['50%']['CN'])} ({int(wmh_summary['25%']['CN'])}-{int(wmh_summary['75%']['CN'])})",
        f"{int(wmh_summary['50%']['MCI'])} ({int(wmh_summary['25%']['MCI'])}-{int(wmh_summary['75%']['MCI'])})",
        f"{round(pval_wmh_diff_cn_vs_mci, 3)}"],
    "Fazekas score binned n (%)": ["", "", round(p_wmh, 3)],
    "WMH": [f"{ct_wmh.iloc[0, 0]} ({round(ct_wmh.iloc[0, 0]/cu_num * 100, 1)}%)", f"{ct_wmh.iloc[0, 1]} ({round(ct_wmh.iloc[0, 1] / mci_num * 100, 1)}%)", ""],
    "no WMH": [f"{ct_wmh.iloc[1, 0]} ({round(ct_wmh.iloc[1, 0]/cu_num * 100, 1)}%)", f"{ct_wmh.iloc[1, 1]} ({round(ct_wmh.iloc[1, 1] / mci_num * 100, 1)}%)", ""],}

# Create the summary DataFrame
df_summary = pd.DataFrame(data = summary_data)

# Print the summary DataFrame
df_table1_bis = df_summary.T
df_table1_bis.columns = [f"CU, n = {cu_num}", f"MCI, n = {mci_num}", "p-value"]
df_table1_bis.to_csv(RES_DIR / "table_1_group_together.csv")
df_table1_bis

## Figures 

In [ ]:
# Plot the boxplot
fig, axs = plt.subplots(2, 2, figsize = (10, 8))
plot_obs(df_max_comp_score_CN, "comp_score", axs[0, 0])
plot_obs(df_max_comp_score_mci, "comp_score", axs[0, 1]) 
axs[0, 0].set_xlabel("")
axs[0, 1].set_xlabel("")
axs[0, 0].set_ylabel("GFS")
axs[0, 1].set_ylabel("GFS")
axs[0, 0].set_title("CU WMH")
axs[0, 1].set_title("MCI WMH")
plot_random_comparison(df_max_comp_score_boxplot_compare_rand_cn, "comp_score", axs[1, 0])
plot_random_comparison(df_max_comp_score_boxplot_compare_rand_mci, "comp_score", axs[1, 1])
axs[1, 0].set_xlabel("Model type")
axs[1, 1].set_xlabel("Model type")
axs[1, 0].set_ylabel("GFS")
axs[1, 1].set_ylabel("GFS")
axs[1, 0].legend(loc = "upper right", bbox_to_anchor=(1.02, 1.175))
axs[1, 1].legend(loc = "upper right", bbox_to_anchor=(1.02, 1.175))
axs = axs.flat

for n, ax in enumerate(axs):
    
    ax.text(-0.1, 1.1, string.ascii_uppercase[n], transform=ax.transAxes, 
            size=20, weight='bold')
fig.tight_layout()

In [ ]:
plot_CN_mci_comparison("comp_score", df_max_boxplot_compare)

In [ ]:
plot_save_fig2(df_max_comp_score_a_cn_mmse, df_max_comp_score_a_mci_mmse, df_max_comp_score_G_cn_mmse, df_max_comp_score_G_mci_mmse) # run these

## Supplementary

### Supplementary Tables

In [ ]:
from itertools import combinations


def results_combo(df, all_pairs, rand=False):
    # Create a dictionary to store the results
    results = {}
    if not rand:
        # Loop through the pairs and perform wilcoxon test to see if model performance is different among all possible pairs
        for pair in all_pairs:
            group1 = df[df["Model_type"] == pair[0]]
            group2 = df[df["Model_type"] == pair[1]]
            # Do it for all observables (PCC; 1-KSD; global functional score)
            # Note: Reported as Wilcoxon T test since Scipy.stats.wilcoxon() method reports the T value and not the W value
            t_statistic_comp_score, p_value_comp_score = stats.wilcoxon(
                group1["comp_score"], group2["comp_score"]
            )
            # Save everything in a results dictionary
            results[f"{pair[0]} vs {pair[1]}"] = {
                "t_statistic_comp_score": t_statistic_comp_score,
                "p_value_comp_score": p_value_comp_score,
            }
    # If we are dealing with random, we need to change the column that we use to get the model names, the rest stays the same
    else:
        for pair in all_pairs:
            group1 = df[df["Model_type_composite"] == pair[0]]
            group2 = df[df["Model_type_composite"] == pair[1]]
            t_statistic_comp_score, p_value_comp_score = stats.wilcoxon(
                group1["comp_score"], group2["comp_score"]
            )
            results[f"{pair[0]} vs {pair[1]}"] = {
                "t_statistic_comp_score": t_statistic_comp_score,
                "p_value_comp_score": p_value_comp_score,
            }

    return results

df_max_comp_score_boxplot_compare_rand_cn = pd.concat([df_max_rand_compare_cn, df_max_comp_score_CN])
df_max_comp_score_boxplot_compare_rand_mci = pd.concat([df_max_rand_compare_mci, df_max_comp_score_mci])

# Create a column to have the composite name of the model + if it is random or non-random in the same cell
df_max_comp_score_boxplot_compare_rand_cn["Model_type_composite"] = (
    df_max_comp_score_boxplot_compare_rand_cn["Model_type"]
    + " "
    + df_max_comp_score_boxplot_compare_rand_cn["Random"]
)
df_max_comp_score_boxplot_compare_rand_mci["Model_type_composite"] = (
    df_max_comp_score_boxplot_compare_rand_mci["Model_type"]
    + " "
    + df_max_comp_score_boxplot_compare_rand_mci["Random"]
)
# Use itertools.combinations to create a combination of all model types (same for CN and MCI)
model_pairs = list(combinations(df_max_comp_score_boxplot_compare_rand_cn["Model_type"].unique(), 2))
# Create combination of all model types vs. their random counterparts
random_pairs = [
    ("Homo. Not Random", "Homo. Random"),
    ("Hetero. Not Random", "Hetero. Random"),
    ("G-weight Not Random", "G-weight Random"),
]

# Perform all the comparisons both for CU with WMH and for MCI with WMH for non-random vs. baseline and vs. one another...
results_pairs_cn = results_combo(df_max_comp_score_boxplot_compare_rand_cn, model_pairs, rand=False)
results_pairs_mci = results_combo(df_max_comp_score_boxplot_compare_rand_mci, model_pairs, rand=False)
#... and for non-random vs. their random counterpart
results_pairs_cn_rand = results_combo(
    df_max_comp_score_boxplot_compare_rand_cn, random_pairs, rand=True
)
results_pairs_mci_rand = results_combo(
    df_max_comp_score_boxplot_compare_rand_mci, random_pairs, rand=True
)
# Store everythin in dataframes 
df_results_pairs_cn = pd.DataFrame().from_dict(results_pairs_cn).T
df_results_pairs_mci = pd.DataFrame().from_dict(results_pairs_mci).T
df_results_pairs_cn_rand = pd.DataFrame().from_dict(results_pairs_cn_rand).T
df_results_pairs_mci_rand = pd.DataFrame().from_dict(results_pairs_mci_rand).T

# Now we create a summary dataset with quartiles and median for the different observables for CU WMH
df_describe_model_fit_cn_wmh = round(
    df_max_comp_score_boxplot_compare_rand_cn.groupby("Model_type").describe()[
        [
            ("comp_score", "25%"),
            ("comp_score", "50%"),
            ("comp_score", "75%"),
        ]
    ],
    3,
)
# ... and for MCI WMH
df_describe_model_fit_mci_wmh = round(
    df_max_comp_score_boxplot_compare_rand_mci.groupby("Model_type").describe()[
        [
            ("comp_score", "25%"),
            ("comp_score", "50%"),
            ("comp_score", "75%"),
        ]
    ],
    3,
)

# This is just to create a subIndex with the group name for better visualization
df_describe_model_fit_cn_wmh_nicer = pd.concat(
    {"CU with WMH": df_describe_model_fit_cn_wmh}, names=["Group"]
)
df_describe_model_fit_mci_wmh_nicer = pd.concat(
    {"MCI with WMH": df_describe_model_fit_mci_wmh}, names=["Group"]
)
df_describe_model_fit_together = pd.concat(
    [df_describe_model_fit_cn_wmh_nicer, df_describe_model_fit_mci_wmh_nicer]
)
# Here we are going to summarise in just one column median (IQR) for all observables...
df_describe_model_fit_together[("GFS", "Median (IQR)")] = (
    df_describe_model_fit_together[("comp_score", "50%")].astype(str)
    + " ("
    + df_describe_model_fit_together[("comp_score", "25%")].astype(str)
    + "-"
    + df_describe_model_fit_together[("comp_score", "75%")].astype(str)
    + ")"
)
# ... and we get rid of previous columns, as they are now redundant
df_describe_model_fit_final= df_describe_model_fit_together.iloc[:, -3:]
# Now we want to add to the table all the p-values of the comparisons between model fits that were previously calculated
# It is a bit tedious now, as I set them one by one, probably there is a faster way?
# We populate the table twice with the same numbers (col, row) and (row, col), but I feel it's more understandable
# compared to just leaving a lot of empty cells...
# We also change the names of the columns, to be in line with the observables described in the paper
list_obs = [("p-value GFS", "p_value_comp_score"),
            ]

# For different group we need to feed from the specific group dataset
group_dfs = [("CU with WMH", df_results_pairs_cn),
            ("MCI with WMH", df_results_pairs_mci),]

for obs in list_obs:
    # We first create empty columns
    df_describe_model_fit_final[(obs[0], "vs. Base")] = np.nan
    df_describe_model_fit_final[(obs[0], "vs. Disconn.")] = np.nan
    df_describe_model_fit_final[(obs[0], "vs. G-weight")] = np.nan
    df_describe_model_fit_final[(obs[0], "vs. Hetero.")] = np.nan
    df_describe_model_fit_final[(obs[0], "vs. Homo.")] = np.nan

    for df in group_dfs:
        df_describe_model_fit_final.loc[
            [(df[0], "Disconn.")], [(obs[0], "vs. Base")]
        ] = df[1].loc["Base vs Disconn.", obs[1]]
        df_describe_model_fit_final.loc[
            [(df[0], "G-weight")], [(obs[0], "vs. Base")]
        ] = df[1].loc["Base vs G-weight", obs[1]]
        df_describe_model_fit_final.loc[
            [(df[0], "Hetero.")], [(obs[0], "vs. Base")]
        ] = df[1].loc["Base vs Hetero.", obs[1]]
        df_describe_model_fit_final.loc[
            [(df[0], "Homo.")], [(obs[0], "vs. Base")]
        ] = df[1].loc["Base vs Homo.", obs[1]]

        df_describe_model_fit_final.loc[
            [(df[0], "Base")], [(obs[0], "vs. Disconn.")]
        ] = df[1].loc["Base vs Disconn.", obs[1]]
        df_describe_model_fit_final.loc[
            [(df[0], "Base")], [(obs[0], "vs. G-weight")]
        ] = df[1].loc["Base vs G-weight", obs[1]]
        df_describe_model_fit_final.loc[
            [(df[0], "Base")], [(obs[0], "vs. Hetero.")]
        ] = df[1].loc["Base vs Hetero.", obs[1]]
        df_describe_model_fit_final.loc[
            [(df[0], "Base")], [(obs[0], "vs. Homo.")]
        ] = df[1].loc["Base vs Homo.", obs[1]]

        df_describe_model_fit_final.loc[
            [(df[0], "G-weight")], [(obs[0], "vs. Disconn.")]
        ] = df[1].loc["G-weight vs Disconn.", obs[1]]
        df_describe_model_fit_final.loc[
            [(df[0], "Hetero.")], [(obs[0], "vs. Disconn.")]
        ] = df[1].loc["Hetero. vs Disconn.", obs[1]]
        df_describe_model_fit_final.loc[
            [(df[0], "Homo.")], [(obs[0], "vs. Disconn.")]
        ] = df[1].loc["Homo. vs Disconn.", obs[1]]

        df_describe_model_fit_final.loc[
            [(df[0], "Disconn.")], [(obs[0], "vs. G-weight")]
        ] = df[1].loc["G-weight vs Disconn.", obs[1]]
        df_describe_model_fit_final.loc[
            [(df[0], "Disconn.")], [(obs[0], "vs. Hetero.")]
        ] = df[1].loc["Hetero. vs Disconn.", obs[1]]
        df_describe_model_fit_final.loc[
            [(df[0], "Disconn.")], [(obs[0], "vs. Homo.")]
        ] = df[1].loc["Homo. vs Disconn.", obs[1]]

        df_describe_model_fit_final.loc[
            [(df[0], "Hetero.")], [(obs[0], "vs. G-weight")]
        ] = df[1].loc["Hetero. vs G-weight", obs[1]]
        df_describe_model_fit_final.loc[
            [(df[0], "Homo.")], [(obs[0], "vs. G-weight")]
        ] = df[1].loc["Homo. vs G-weight", obs[1]]

        df_describe_model_fit_final.loc[
            [(df[0], "G-weight")], [(obs[0], "vs. Hetero.")]
        ] = df[1].loc["Hetero. vs G-weight", obs[1]]
        df_describe_model_fit_final.loc[
            [(df[0], "G-weight")], [(obs[0], "vs. Homo.")]
        ] = df[1].loc["Homo. vs G-weight", obs[1]]

        df_describe_model_fit_final.loc[
            [(df[0], "Homo.")], [(obs[0], "vs. Hetero.")]
        ] = df[1].loc["Homo. vs Hetero.", obs[1]]

        df_describe_model_fit_final.loc[
            [(df[0], "Hetero.")], [(obs[0], "vs. Homo.")]
        ] = df[1].loc["Homo. vs Hetero.", obs[1]]

# We rearrange the table so that the p-values for an observable are next to the median (IQR) of that observable
cols = df_describe_model_fit_final.columns.tolist()
cols = cols[0:1] + cols[-15:-10] + cols[1:2] + cols[-10:-5] + cols[2:3] + cols[-5:]
df_describe_model_fit_final = df_describe_model_fit_final[cols]

# Better formatting 
all_cols = df_describe_model_fit_final.columns.to_list() 
num = df_describe_model_fit_final._get_numeric_data()
num_cols = num.columns.to_list()
non_num_cols = [col for col in all_cols if col not in num_cols]
num[num.astype(float) < 0.001] = "< 0.001"
df_newly_formatted = pd.concat([num, df_describe_model_fit_final[non_num_cols]], axis = 1)
new_cols = df_newly_formatted.columns.to_list()
new_cols_ordered = new_cols[-3:-2] + new_cols[0:5] + new_cols[-2:-1] + new_cols[5:10] + new_cols[-1:] + new_cols[10:15]
df_newly_formatted = df_newly_formatted[new_cols_ordered]
df_newly_formatted.to_csv(RES_DIR / "table_model_goodness_of_fit.csv")
df_newly_formatted

### Supplementary Figures

In [ ]:
wmh_pre = (
        df_petTOAD_pre["WMH_load_subj_space"] - df_petTOAD_pre["WMH_load_subj_space"].min()
    ) / (
        df_petTOAD_pre["WMH_load_subj_space"].max() - df_petTOAD_pre["WMH_load_subj_space"].min()
    )
wmh_post = (
        df_petTOAD["WMH_load_subj_space"] - df_petTOAD["WMH_load_subj_space"].min()
    ) / (
        df_petTOAD["WMH_load_subj_space"].max() - df_petTOAD["WMH_load_subj_space"].min()
    )
plt.figure(figsize=(8,4))
plt.subplot(121)
plt.hist(wmh_pre, bins = 25);
plt.annotate(f"Median: {round(wmh_pre.median(), 2)}\nSkewness: {round(stats.skew(wmh_pre), 2)}", (0.5, 50))
plt.subplot(122)
plt.hist(wmh_post, bins = 25);
plt.annotate(f"Median: {round(wmh_post.median(), 2)}\nSkewness: {round(stats.skew(wmh_post), 2)}", (0.5, 38))
plt.savefig(FIG_DIR / "distribution_pre_post_normalized_WMH.png")

## Other old code

In [ ]:
# print("The best (b, w) for CN subjects are as follows:")
# print(f"Homogeneous a-weighted model: {best_fit_b_CN_a, best_fit_w_CN_a}")
# print(f"Homogeneous G-weighted model: {best_fit_b_CN_G, best_fit_w_CN_G}")
# print(f"Heterogeneous a-weighted model: {best_fit_b_CN_het, best_fit_w_CN_het}")
# print("######################################################################")
# print("The best b and w for MCI subjects are as follows:")
# print(f"Homogeneous a-weighted model: {best_fit_b_mci_a, best_fit_w_mci_a}")
# print(f"Homogeneous G-weighted model: {best_fit_b_mci_G, best_fit_w_mci_G}")
# print(f"Heterogeneous a-weighted model: {best_fit_b_mci_het, best_fit_w_mci_het}")

# dict_summary_best_w_b = {
#     "": ["b", "w", "b", "w"],
#     "Homogeneous a-weighted": [
#         f"{best_fit_b_CN_a}",
#         f"{best_fit_w_CN_a}",
#         f"{best_fit_b_mci_a}",
#         f"{best_fit_w_mci_a}",
#     ],
#     "G-weighted model": [
#         f"{best_fit_b_CN_G}",
#         f"{best_fit_w_CN_G}",
#         f"{best_fit_b_mci_G}",
#         f"{best_fit_w_mci_G}",
#     ],
#     "Heterogeneous a-weighted model": [
#         f"{best_fit_b_CN_het}",
#         f"{best_fit_w_CN_het}",
#         f"{best_fit_b_mci_het}",
#         f"{best_fit_w_mci_het}",
#     ],
# }
# df_summary_best_w_b = pd.DataFrame.from_dict(dict_summary_best_w_b).T
# df_summary_best_w_b.columns = ["CU WMH", "", "MCI WMH", ""]
# df_summary_best_w_b.to_csv(RES_DIR / "table2.csv")
# df_summary_best_w_b

In [ ]:
# # Plot the boxplot
# fig, axs = plt.subplots(2, 2, figsize = (10, 8))
# plot_obs(df_boxplot_compare_CN, "comp_score", axs[0, 0])
# plot_obs(df_boxplot_compare_mci, "comp_score", axs[0, 1])
# axs[0, 0].set_xlabel("")
# axs[0, 1].set_xlabel("")
# axs[0, 0].set_ylabel("GFS")
# axs[0, 1].set_ylabel("GFS")
# axs[0, 0].set_title("CU WMH")
# axs[0, 1].set_title("MCI WMH")
# plot_random_comparison(df_boxplot_compare_rand_cn, "comp_score", axs[1, 0])
# plot_random_comparison(df_boxplot_compare_rand_mci, "comp_score", axs[1, 1])
# axs[1, 0].set_xlabel("Model type")
# axs[1, 1].set_xlabel("Model type")
# axs[1, 0].set_ylabel("GFS")
# axs[1, 1].set_ylabel("GFS")
# axs[1, 0].legend(loc = "upper right", bbox_to_anchor=(1.02, 1.175))
# axs[1, 1].legend(loc = "upper right", bbox_to_anchor=(1.02, 1.175))
# axs = axs.flat

# for n, ax in enumerate(axs):
    
#     ax.text(-0.1, 1.1, string.ascii_uppercase[n], transform=ax.transAxes, 
#             size=20, weight='bold')
# fig.tight_layout()

In [ ]:
# # Plot the model performance between groups
# fig = plt.figure(figsize=(6,4))
# plot_CN_mci_comparison("comp_score", df_boxplot_compare)
# plt.xlabel("Model type")
# plt.ylabel("GFS")
# plt.legend()
# plt.savefig(FIG_DIR / "Figure4.png")

In [ ]:
# # Plot a histogram depicting how the distributions of a and G vary between groups
# list_a_homo_cn = []
# list_G_cn = []
# list_a_het_cn = []
# list_wmh_cn = []

# list_a_homo_mci = []
# list_G_mci = []
# list_a_het_mci = []
# list_wmh_mci = []

# for subj in subjs_to_sim[:-1]:
#     wmh = wmh_dict[subj]
#     #node_damage = get_node_damage_ext(subj)

#     if subj in CN_WMH:
#         list_wmh_cn.append(wmh)
#         a_homo = -0.02 + best_fit_w_CN_a * wmh + best_fit_b_CN_a
#         G = 2.0 + best_fit_w_CN_G * wmh + best_fit_b_CN_G
#         #a_het = np.array([round(-0.02 + best_fit_w_CN_het*(1-n) + best_fit_b_CN_het, 5) for n in node_damage])
#         list_a_homo_cn.append(a_homo)
#         list_G_cn.append(G)
#         #list_a_het_cn.append(a_het)
        
#     elif subj in MCI_WMH:
#         list_wmh_mci.append(wmh)
#         a_homo = -0.02 + best_fit_w_mci_a * wmh + best_fit_b_mci_a
#         G = 2.0 + best_fit_w_mci_G * wmh + best_fit_b_mci_G
#         #a_het = np.array([round(-0.02 + best_fit_w_mci_het*(1-n) + best_fit_b_mci_het, 5) for n in node_damage])
#         list_a_homo_mci.append(a_homo)
#         list_G_mci.append(G)
#         #list_a_het_mci.append(a_het)


# print(f"The min. value of a for CU with WMH is {round(min(list_a_homo_cn), 3)}")
# print(f"The max. value of a for CU with WMH is {round(max(list_a_homo_cn), 3)}")
# # print(f"The min. value of a heterogeneous for CU with WMH is {round(min([min(node) for node in list_a_het_cn]), 3)}")
# # print(f"The max. value of a heterogeneous for CU with WMH is {round(max([max(node) for node in list_a_het_cn]), 3)}")
# print(f"The min. value of G for CU with WMH is {round(min(list_G_cn), 3)}")
# print(f"The max. value of G for CU with WMH is {round(max(list_G_cn), 3)}")

# print(f"The min. value of a homogeneous for MCI with WMH is {round(min(list_a_homo_mci), 3)}")
# print(f"The max. value of a homogeneous for MCI with WMH is {round(max(list_a_homo_mci), 3)}")
# # print(f"The min. value of a heterogeneous for MCI with WMH is {round(min([min(node) for node in list_a_het_mci]), 3)}")
# # print(f"The max. value of a heterogeneous for MCI with WMH is {round(max([max(node) for node in list_a_het_mci]), 3)}")
# print(f"The min. value of G for MCI with WMH is {round(min(list_G_mci), 3)}")
# print(f"The max. value of G for MCI with WMH is {round(max(list_G_mci), 3)}")

In [ ]:
# plot how the histplot changes
    # fig, axs = plt.subplots(2, 2, figsize=(10, 8))
    # # plt.hist(list_a_homo_cn, weights=np.ones_like(list_a_homo_cn) / len(list_a_homo_cn), alpha = 0.4, label = "CU WMH");
    # axs[0, 0].hist(
    #     list_a_homo_mci,
    #     weights=np.ones_like(list_a_homo_mci) / len(list_a_homo_mci),
    #     alpha=0.4,
    #     bins=15,
    #     label="MCI WMH",
    # )

    # axs[0, 0].set_ylabel("Frequency")
    # axs[0, 0].set_xlabel(r"$a$")
    # axs[0, 0].set_xlim(-0.041, -0.015)
    # axs[0, 0].axvline(x=-0.02, color="orange", lw=2, linestyle=":", label="CU no WMH")
    # axs[0, 0].arrow(
    #     -0.02,
    #     0.25,
    #     -0.005,
    #     0,
    #     length_includes_head=False,
    #     width=0.01,
    #     head_length=0.002,
    #     head_width=0.015,
    #     fill=False,
    #     linestyle="-",
    #     lw=0.8,
    #     edgecolor="black",
    # )

    # axs[0, 1].hist(
    #     list_G_mci,
    #     weights=np.ones_like(list_G_mci) / len(list_G_mci),
    #     alpha=0.4,
    #     bins=15,
    #     label="MCI WMH",
    # )
    # axs[0, 1].set_xlabel(r"$G$")
    # axs[0, 1].set_ylabel("Frequency")
    # axs[0, 1].set_xlim(0.9, 2.1)
    # axs[0, 1].axvline(x=2.0, color="orange", lw=2, linestyle=":", label = "CU WMH")
    # axs[0, 1].arrow(
    #     2,
    #     0.25,
    #     -0.25,
    #     0,
    #     length_includes_head=False,
    #     width=0.01,
    #     head_length=0.1,
    #     head_width=0.015,
    #     fill=False,
    #     linestyle="-",
    #     lw=0.8,
    #     edgecolor="black",
    # )


In [ ]:
# # PCC and 1 - KSD in MCI WMH
# fig, axs = plt.subplots(2, 2, figsize = (15, 8))
# plot_obs(df_boxplot_compare_mci, "fc_pearson", axs[0, 0])
# plot_obs(df_boxplot_compare_mci, "phfcd_ks", axs[0, 1])
# plot_random_comparison(df_boxplot_compare_rand_mci, "fc_pearson", axs[1, 0])
# plot_random_comparison(df_boxplot_compare_rand_mci, "phfcd_ks", axs[1, 1])

# axs[0, 0].set_xlabel("")
# axs[0, 1].set_xlabel("")
# axs[0, 0].set_ylabel("PCC")
# axs[0, 1].set_ylabel("1 - KSD")
# axs[1, 0].set_xlabel("Model type")
# axs[1, 1].set_xlabel("Model type")
# axs[1, 0].set_ylabel("PCC")
# axs[1, 1].set_ylabel("1 - KSD")
# axs = axs.flat

# for n, ax in enumerate(axs):
    
#     ax.text(-0.1, 1.1, string.ascii_uppercase[n], transform=ax.transAxes, 
#             size=20, weight='bold')
# fig.tight_layout()
# fig.suptitle("MCI WMH")

In [ ]:

# If you want / need to do also PCC and 1-phFCD comparison us plt.subplot() and change the function to also accept axis

# plot_CN_mci_comparison("fc_pearson", axs[0])
# axs[0].set_ylabel("PCC")
# axs[0].set_xlabel("Model Type")
# axs[0].legend(["CU WMH", "MCI WMH"])
# leg0 = axs[0].get_legend()
# leg0.legendHandles[1].set_color('C0')
# leg0.legendHandles[1].set_edgecolor('k')
# plot_CN_mci_comparison("phfcd_ks", axs[1])
# axs[1].set_ylabel("1 - KSD")
# axs[1].set_xlabel("Model Type")
# axs[1].legend(["CU WMH", "MCI WMH"])
# leg1 = axs[1].get_legend()
# leg1.legend_handles[1].set_color('C0')
# leg1.legend_handles[1].set_edgecolor('k')
# plot_CN_mci_comparison("comp_score", axs[2])
# axs[2].set_ylabel("GFS")
# axs[2].set_xlabel("Model Type")
# axs[2].legend(["CU WMH", "MCI WMH"])
# leg2 = axs[2].get_legend()
# leg2.legend_handles[1].set_color('C0')
# leg2.legend_handles[1].set_edgecolor('k')
# fig.tight_layout()

In [ ]:
# # PCC and 1 - KSD in CU WMH
# fig, axs = plt.subplots(2, 2, figsize = (20, 16))
# plot_obs(df_boxplot_compare_CN, "fc_pearson", axs[0, 0])
# plot_obs(df_boxplot_compare_CN, "phfcd_ks", axs[0, 1])
# plot_random_comparison(df_boxplot_compare_rand_cn, "fc_pearson", axs[1, 0])
# plot_random_comparison(df_boxplot_compare_rand_cn, "phfcd_ks", axs[1, 1])

# axs[0, 0].set_xlabel("")
# axs[0, 1].set_xlabel("")
# axs[0, 0].set_ylabel("PCC")
# axs[0, 1].set_ylabel("1 - KSD")
# axs[1, 0].set_xlabel("Model type")
# axs[1, 1].set_xlabel("Model type")
# axs[1, 0].set_ylabel("PCC")
# axs[1, 1].set_ylabel("1 - KSD")
# axs = axs.flat

# for n, ax in enumerate(axs):
    
#     ax.text(-0.1, 1.1, string.ascii_uppercase[n], transform=ax.transAxes, 
#             size=20, weight='bold')
# fig.tight_layout()

In [ ]:
# df_max_comp_score_a_cn = df_max_comp_score_a[df_max_comp_score_a["PTID"].isin(CN_WMH)]
# df_max_comp_score_a_mci = df_max_comp_score_a[df_max_comp_score_a["PTID"].isin(MCI_WMH)]

# # Create a JointGrid for the scatter plots and histograms
# grid1 = sns.JointGrid(height=6, ratio=3)

# # Plot the scatter plots on the grid
# sns.regplot(data=df_max_comp_score_a_cn, x="wmh_load", y="a", ax=grid1.ax_joint, label="CU WMH")
# sns.regplot(data=df_max_comp_score_a_mci, x="wmh_load", y="a", ax=grid1.ax_joint, label="MCI WMH")

# # Plot histograms on the top and left sides of the grid
# sns.histplot(data=df_max_comp_score_a_cn, x="wmh_load", ax=grid1.ax_marg_x, kde=True)
# sns.histplot(data=df_max_comp_score_a_mci, x="wmh_load", ax=grid1.ax_marg_x, kde=True, color = "tab:orange")
# sns.histplot(data=df_max_comp_score_a_cn, y="a", ax=grid1.ax_marg_y, kde=True)
# sns.histplot(data=df_max_comp_score_a_mci, y="a", ax=grid1.ax_marg_y, kde=True, color = "tab:orange")

# # Set labels for the scatter plot and legends
# grid1.set_axis_labels("WMH Load", "a")
# grid1.ax_joint.legend()
# ax = grid1.fig.axes[0]
# ax.text(-0.1, 1.1, "C", transform=ax.transAxes, 
#             size=20, weight='bold')